<a href="https://colab.research.google.com/github/alexlwn123/WikipediaSpeedrun/blob/master/Info_Retrieval_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
!pip install wikipedia-api
import wikipediaapi as wikipedia
from nltk.corpus import stopwords
import string
from sklearn.metrics.pairwise import linear_kernel
import concurrent.futures
from hashlib import md5
import sys
import random

  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-cp37-none-any.whl size=13462 sha256=df5c51173dadc269d9baa217adf5dce33d4211740fa3bd0765e5e5e90d3da836
  Stored in directory: /root/.cache/pip/wheels/bf/40/42/ba1d497f3712281b659dd65b566fc868035c859239571a725a
Successfully built wikipedia-api


In [ ]:
import nltk
nltk.download('stopwords')
twenty = fetch_20newsgroups()
wiki = wikipedia.Wikipedia('en')
stopWords = stopwords.words('english') 

documents = []
titles = {}


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def get_page(title):
    if ':' in link:
        return
    new_page = wiki.page(link)
    if new_page.exists():
        new_doc = clean_doc(new_page.summary)
        return (link, new_doc)

In [ ]:
def clean_doc(document):
    return (document, [word for line in document.split('\n') for word in line.translate(str.maketrans('','', string.punctuation)).lower().split()])

In [ ]:
def add_link(link):
    if ':' in link:
        return
    new_page = wiki.page(link)
    try:
        # if new_page.exists():
        #     # summ = new_page.summary
        #     #titles[md5(summ.encode())] = link
        #     # return (link, summ)
        return (link, new_page.summary)
    except:
         return
        #clean_doc(new_page.summary)
       

In [ ]:
def donzeed(checked, link):
    for x in checked:
        if link.startswith(x):
            return True
    return False
def add_links(curr_page):
    with concurrent.futures.ThreadPoolExecutor() as executor:            
        futures = []
        results = {}
        checked = []
        links = [x for x in list(curr_page.links.keys()) if x not in titles.values()]

        for link in links[:min(90, len(links))]:
            if donzeed(checked, link):
                 continue 
            #print(link, end =' ')
            #add_link(link)
            checked.append(link)
            futures.append(executor.submit(add_link, link))

        for future in concurrent.futures.as_completed(futures):
            res = future.result()
            if not res:
                continue
            title, summary = res[0], res[1]
            documents.append(summary)
            titles[md5(summary.encode()).digest()] = title
        



In [ ]:
def update_tfidfs(target):

    tfidf = TfidfVectorizer().fit_transform(documents)
    
    cosine_similarities = linear_kernel(tfidf[0:1], tfidf).flatten()   
    
    return cosine_similarities

In [ ]:
start = "mario"
target = "WWII"

import cProfile
import pstats
import timeit

start_time = timeit.default_timer()

start_page, target_page = wiki.page(start), wiki.page(target)
target = target_page.title
# targets = [target]
targets = ['WWII', 'World War II', 'The Second World War', 'WW2']
#targets = ["SCP Foundation"]
#targets = ["Beyoncé"]

def contains_any(targets, links):
    for target in targets:
        if target in links:
            return True
    return False

print(start, '-->', target)

if not start_page.exists() or not target_page.exists():
        print('Start and/or Target page does not exist.')
        print('Exiting...')
        sys.exit(1)

titles = {}
documents = []

titles[md5(target_page.summary.encode()).digest()] = target
titles[md5(start_page.summary.encode()).digest()] = start
documents.append(target_page.summary)
documents.append(start_page.summary)
 


curr_page = start_page

twenty = fetch_20newsgroups()
#print('twenty', type(twenty.data[0]))

path = [start]
print('Before', timeit.default_timer() - start_time)

last = start
#while target not in curr_page.links:
while not contains_any(targets, curr_page.links):
    if timeit.default_timer() - start_time > 100:
        print('Timeout: > 100s')
        print('Exiting...')
        sys.exit(1)
    if not curr_page.exists():
        print(curr_page, "does not exist")
        sys.exit(1)

    print(curr_page.title,  timeit.default_timer() - start_time)
    add_links(curr_page)
    print('Added Links', timeit.default_timer() - start_time)
    cosines = update_tfidfs(target)
    print('Updated tfidfs',  timeit.default_timer() - start_time)
    related_docs_indices = cosines.argsort()[:-5:-1]
    print()
# print('cosines', cosines)
    print('indx', related_docs_indices)
    print('top', cosines[related_docs_indices])
    best = titles[md5(documents[related_docs_indices[1]].encode()).digest()]
    print('Title:', best)
    if last != best:
        path.append(best)
    else:
        best = titles[md5(documents[related_docs_indices[2]].encode()).digest()]
        path.append(best)
    last = best
    curr_page = wiki.page(best)
    print('Done',  timeit.default_timer() - start_time)

path.append(target)
print('\nPath...')
print(' --> '.join(path))
print('Total',  timeit.default_timer() - start_time)



mario --> WWII
Before 6.067649393000011
Mario 7.044223074000001
Added Links 17.15273493899997
Updated tfidfs 17.17381834699995

indx [ 0 13 49 46]
top [1.         0.36175756 0.31402152 0.30665044]
Title: Association football
Done 17.17756128999997

Path...
mario --> Association football --> WWII
Total 18.780584567999995
